In [ ]:
#This one for Memba compartivie Networks 

import torch
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from loader import *


#from models.Unet_Nested.UNet_Nested import UNet_Nested 
#from models.Unet.unet_model import UNet
#from models.CENet.cenet import CE_Net

#from models.transnorm.vit_seg_modeling import VisionTransformer as ViT_seg
#from models.transnorm.vit_seg_modeling import CONFIGS as CONFIGS_ViT_seg

#from models.AttUnet.AttUnet import R2U_Net, AttU_Net, R2AttU_Net
#from models.SegNet.segnet import SegNet, SegResNet 
#from models.RefineNet.RefineNet import get_refinenet 
#from models.GLNet.glnet import GLNet
#from models.UViT.UViT import UViT
from models.UViT.UViT_modified import UViT

from engine import *
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # "0, 1, 2, 3"

from utils import *
from configs.config_setting_others import setting_config

import warnings
warnings.filterwarnings("ignore")



def main(config):

    print('#----------Creating logger----------#')
    sys.path.append(config.work_dir + '/')
    log_dir = os.path.join(config.work_dir, 'log')
    checkpoint_dir = os.path.join(config.work_dir, 'checkpoints')
    resume_model = os.path.join(checkpoint_dir, 'latest.pth')
    outputs = os.path.join(config.work_dir, 'outputs')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(outputs):
        os.makedirs(outputs)

    global logger
    logger = get_logger('train', log_dir)

    log_config_info(config, logger)





    print('#----------GPU init----------#')
    set_seed(config.seed)
    gpu_ids = [0]# [0, 1, 2, 3]
    torch.cuda.empty_cache()





    print('#----------Preparing dataset----------#')
    train_dataset = RemSemLoader(root_dir = config.data_path, train = True)
    train_loader = DataLoader(train_dataset,
                                batch_size=config.batch_size, 
                                shuffle=True,
                                pin_memory=True,
                                num_workers=config.num_workers)
    val_dataset = RemSemLoader(root_dir = config.data_path, train = False)
    val_loader = DataLoader(val_dataset,
                                batch_size=1,
                                shuffle=False,
                                pin_memory=True, 
                                num_workers=config.num_workers,
                                drop_last=True)
    test_dataset = RemSemLoader(root_dir = config.data_path, train = False, test = True)
    test_loader = DataLoader(test_dataset,
                                batch_size=1,
                                shuffle=False,
                                pin_memory=True, 
                                num_workers=config.num_workers,
                                drop_last=True)




    print('#----------Prepareing Models----------#')
    model_cfg = config.model_config
    





    
    #model = UNet_Nested(n_classes=model_cfg['num_classes'], in_channels=model_cfg['input_channels'])
    #model = UNet(n_classes=model_cfg['num_classes'], n_channels=model_cfg['input_channels'])
    #model = CE_Net(num_classes=model_cfg['num_classes'], num_channels=model_cfg['input_channels'])
    #model = R2U_Net(output_ch=model_cfg['num_classes'], img_ch=model_cfg['input_channels'])
    
    
    #model = AttU_Net(output_ch=model_cfg['num_classes'], img_ch=model_cfg['input_channels'])
    #model = R2AttU_Net(output_ch=model_cfg['num_classes'], img_ch=model_cfg['input_channels'])

    #model = SegNet(num_classes=model_cfg['num_classes'], in_channels=model_cfg['input_channels'])
    #model = SegResNet(num_classes=model_cfg['num_classes'], in_channels=model_cfg['input_channels'])

    #model = get_refinenet(input_size=config.input_size_h, in_channel=model_cfg['input_channels'],  num_classes=model_cfg['num_classes'], pretrained=False)
    
    ####---------  Transnorm  based  TransNorm model ------------####
    # vit_name = 'R50-ViT-B_16'
    # vit_patches_size = 16
    # img_size = config.input_size_h
    # config_vit = CONFIGS_ViT_seg['R50-ViT-B_16']
    # config_vit.n_classes = model_cfg['num_classes']
    # config_vit.n_skip = 3
    # if vit_name.find('R50') != -1:
    #     config_vit.patches.grid = (int(img_size / vit_patches_size), int(img_size / vit_patches_size))
    # model = ViT_seg(config_vit, img_size=img_size, num_classes=config_vit.n_classes).to('cuda')
    


   

    ###-----torvision library based DeepLab and FCN segmentation model imeplementation---###
    # import torchvision.models.segmentation as segmentation



    # class CustomDeepLabV3(torch.nn.Module):
    #     def __init__(self, num_classes, input_channels=3):
    #         super(CustomDeepLabV3, self).__init__()
    #         # Load the pre-trained DeepLabv3 model
    #         self.model = segmentation.deeplabv3_resnet50(pretrained=False)
    #         #self.model = segmentation.deeplabv3_resnet101(pretrained=False)
    #         self.model.backbone.conv1 = torch.nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    #         self.model.classifier[4] = torch.nn.Conv2d(256, num_classes, kernel_size=1)
    
    #     def forward(self, x):
    #         return torch.sigmoid(self.model(x)['out'])

    # num_classes = model_cfg['num_classes']  
    # input_channels = model_cfg['input_channels']
    
    # model = CustomDeepLabV3(num_classes=num_classes, input_channels=input_channels)


    # class CustomDeepLabV3MobileNet(torch.nn.Module):
    #     def __init__(self, num_classes, input_channels=3):
    #         super(CustomDeepLabV3MobileNet, self).__init__()
    #         self.model = segmentation.deeplabv3_mobilenet_v3_large(pretrained=False)
    #         backbone = self.model.backbone
    #         first_conv = backbone['0'] 
    #         first_conv[0] = torch.nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding=1, bias=False)
    #         self.model.backbone = backbone
    #         self.model.classifier[4] = torch.nn.Conv2d(256, num_classes, kernel_size=1)
    
    #     def forward(self, x):
    #         return torch.sigmoid(self.model(x)['out'])

    
    # num_classes = model_cfg['num_classes']   
    # input_channels = model_cfg['input_channels']
    # model = CustomDeepLabV3MobileNet(num_classes=num_classes, input_channels=input_channels)


    # class CustomFCN(torch.nn.Module):
    #     def __init__(self, backbone_type='resnet50', num_classes=1, input_channels=3):
    #         super(CustomFCN, self).__init__()
    #         # Load the appropriate FCN model
    #         if backbone_type == 'resnet50':
    #             self.model = segmentation.fcn_resnet50(pretrained=False)
    #         elif backbone_type == 'resnet101':
    #             self.model = segmentation.fcn_resnet101(pretrained=False)
    #         else:
    #             raise ValueError("backbone_type must be 'resnet50' or 'resnet101'")
    
            
    #         self.model.backbone.conv1 = torch.nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    #         self.model.classifier[4] = torch.nn.Conv2d(512, num_classes, kernel_size=1)
    
    #     def forward(self, x):
    #         return torch.sigmoid(self.model(x)['out'])

    # num_classes = model_cfg['num_classes']   
    # input_channels = model_cfg['input_channels']
    # model = CustomFCN(backbone_type='resnet50', num_classes=num_classes, input_channels=input_channels)


    
    #model = GLNet(n_ch=model_cfg['input_channels'],  patch_height=config.input_size_h, patch_width=config.input_size_w, num_class=model_cfg['num_classes'])
    in_bands = model_cfg['input_channels']  # Example value for in_bands
    N = [3, 6, 9, 12]  # Example value for N
    def get_uvit(in_channels,num_classes,in_bands, N,type,img_size=224):
    return UViT(
        in_channels=in_channels,
        out_channels=num_classes,
        in_bands = in_bands,
        img_size=img_size,
        n=N,
        freeze_encoder=False,
        type = type,
        ckpt_path = "/users/lalit47/gnet_seg/models/Prithvi_100M.pt",
        cfg_path = "/users/lalit47/gnet_seg/models/config.yaml"
    ).to('cpu')
    model = get_uvit(in_channels = model_cfg['input_channels'], num_classes = = model_cfg['num_classes']), in_bands, N, type, img_size = config.input_size_h)
    
    model = torch.nn.DataParallel(model.cuda(), device_ids=gpu_ids, output_device=gpu_ids[0])

   
    
    






    print('#----------Prepareing loss, opt, sch and amp----------#')
    criterion = config.criterion
    optimizer = get_optimizer(config, model)
    scheduler = get_scheduler(config, optimizer)
    scaler = GradScaler()





    print('#----------Set other params----------#')
    min_loss = 999
    start_epoch = 1
    min_epoch = 1




   #=-----------------To Resume the model---------------#
    # if os.path.exists(resume_model):
    #     print('#----------Resume Model and Other params----------#')
    #     checkpoint = torch.load(resume_model, map_location=torch.device('cpu'))
    #     model.module.load_state_dict(checkpoint['model_state_dict'])
    #     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    #     scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    #     saved_epoch = checkpoint['epoch']
    #     start_epoch += saved_epoch
    #     min_loss, min_epoch, loss = checkpoint['min_loss'], checkpoint['min_epoch'], checkpoint['loss']

    #     log_info = f'resuming model from {resume_model}. resume_epoch: {saved_epoch}, min_loss: {min_loss:.4f}, min_epoch: {min_epoch}, loss: {loss:.4f}'
    #     logger.info(log_info)





    print('#----------Training----------#')
    for epoch in range(start_epoch, config.epochs + 1):

        torch.cuda.empty_cache()

        train_one_epoch(
            train_loader,
            model,
            criterion,
            optimizer,
            scheduler,
            epoch,
            logger,
            config,
            scaler=scaler
        )

        loss = val_one_epoch(
                val_loader,
                model,
                criterion,
                epoch,
                logger,
                config
            )


        if loss < min_loss:
            torch.save(model.module.state_dict(), os.path.join(checkpoint_dir, 'best.pth'))
            min_loss = loss
            min_epoch = epoch

        torch.save(
            {
                'epoch': epoch,
                'min_loss': min_loss,
                'min_epoch': min_epoch,
                'loss': loss,
                'model_state_dict': model.module.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
            }, os.path.join(checkpoint_dir, 'latest.pth')) 

    if os.path.exists(os.path.join(checkpoint_dir, 'best.pth')):
        print('#----------Testing----------#')
        best_weight = torch.load(config.work_dir + 'checkpoints/best.pth', map_location=torch.device('cpu'))
        model.module.load_state_dict(best_weight)
        loss = test_one_epoch(
                test_loader,
                model,
                criterion,
                logger,
                config,
            )
        os.rename(
            os.path.join(checkpoint_dir, 'best.pth'),
            os.path.join(checkpoint_dir, f'best-epoch{min_epoch}-loss{min_loss:.4f}.pth')
        )      


if __name__ == '__main__':
    config = setting_config
    main(config)